In [2]:
%matplotlib inline

import numpy  as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import antea.mcsim.phantom as ph

# Create a NEMA-like phantom
Note we will sample to a 1 mm resolution.  The phantom consists of:

- 4 radioactive spheres of radii 4, 6.5, 8.5, and 11 mm
- 2 cold spheres of radii 14, 18.5 mm
- a cold central cylinder of radius 20 mm and z-length of 100 mm
- all spheres placed at a radial distance of 114.4 mm from the center of the volume, centered axially
- "radioactive" elements have 4 times the activity of "cold" elements 

### Construct the phantom

In [ ]:
# Create the main volume.
nema = ph.phantom(180,180,180)

# Create the different components of the phantom.
sph_r1 = ph.create_sphere(4)*3
sph_r2 = ph.create_sphere(6.5)*3
sph_r3 = ph.create_sphere(8.5)*3
sph_r4 = ph.create_sphere(11)*3

sph_c1 = ph.create_sphere(14.0)*-1
sph_c2 = ph.create_sphere(18.5)*-1

cyl = ph.create_cylinder(22,20)*-1
bg  = ph.create_sphere(85)*1

# Position them in the main volume.
r_pos = 50

xoff = np.rint(r_pos).astype('int'); yoff = 0
nema.add_to_vol(sph_c2,xoff,yoff,0)
xoff = np.rint(np.cos(np.pi/3)*r_pos).astype('int'); yoff = np.rint(np.sin(np.pi/3)*r_pos).astype('int')
nema.add_to_vol(sph_r1,xoff,yoff,0)
xoff = np.rint(np.cos(2*np.pi/3)*r_pos).astype('int'); yoff = np.rint(np.sin(2*np.pi/3)*r_pos).astype('int')
nema.add_to_vol(sph_r2,xoff,yoff,0)
xoff = np.rint(np.cos(np.pi)*r_pos).astype('int'); yoff = np.rint(np.sin(np.pi)*r_pos).astype('int')
nema.add_to_vol(sph_r3,xoff,yoff,0)
xoff = np.rint(np.cos(4*np.pi/3)*r_pos).astype('int'); yoff = np.rint(np.sin(4*np.pi/3)*r_pos).astype('int')
nema.add_to_vol(sph_r4,xoff,yoff,0)
xoff = np.rint(np.cos(5*np.pi/3)*r_pos).astype('int'); yoff = np.rint(np.sin(5*np.pi/3)*r_pos).astype('int')
neam.add_to_vol(sph_c1,xoff,yoff,0)

nema.add_to_vol(vol,cyl,0,0,0)
nema.add_to_vol(vol,bg,0,0,0)

## Plot and save the phantom

In [ ]:
# Get the volume and plot 
vol = nema.get_volume()
plt.imshow(np.sum(vol[:,:,86:91]/5,axis=2).transpose(),origin='lower')
plt.colorbar()
plt.xlabel("x (mm)")
plt.ylabel("y (mm)")

In [ ]:
# Save the phantom to file.
nema.save("phantom_NEMAlike.npz")

### Generate a 2D profile

In [ ]:
prof = circular_profile(vol[:,:,88],50,100)

In [ ]:
plt.plot(range(len(prof)),prof)

### Write the distribution to a binary file

In [ ]:
# Normalize the volume.
npimg = vol
npimg = npimg.astype('float64') / np.sum(npimg)

In [ ]:
# Flatten and convert to a cumulative distribution.
npimg_flat = npimg.flatten().cumsum()

# Pack the header information (Nx, Ny, Nz, Lx, Ly, Lz)
Nx = npimg.shape[0]
Ny = npimg.shape[1]
Nz = npimg.shape[2]
Lx = float(Nx)
Ly = float(Ny)
Lz = float(Nz)
print("Shape = ({},{},{}); size = ({},{},{})".format(Nx,Ny,Nz,Lx,Ly,Lz))

s_hdr1 = struct.pack('i'*3,Nx,Ny,Nz)
s_hdr2 = struct.pack('f'*3,Lx,Ly,Lz)
s_arr = struct.pack('f'*len(npimg_flat), *npimg_flat)
f = open('phantom_NEMAlike.dat','wb')
f.write(s_hdr1)
f.write(s_hdr2)
f.write(s_arr)
f.close()

In [ ]:
# Perform some checks.
f2 = open('phantom_NEMAlike.dat','rb')
x1 = f2.read(4)
print(struct.unpack('i',x1))

x2 = f2.read(4)
print(struct.unpack('i',x2))

x3 = f2.read(4)
print(struct.unpack('i',x3))

f2.read(4)
f2.read(4)
f2.read(4)

for ii in range(2097151): f2.read(4)
    
print(struct.unpack('f',f2.read(4)))

f2.close()